<a href="https://colab.research.google.com/github/vijayk2000/capstone_project/blob/main/Pinacle_capstone_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install langchain_chroma
%pip install langchain_openai
%pip install langchain_google_genai
%pip install langchain_huggingface
%pip install langchain_community
%pip install FlagEmbedding
!pip install python-dotenv
%pip install pypdf
!pip install selfcheckgpt

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.docstore.document import Document

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Load the files and index them in a vector database


In [12]:
docs= []
for doc in PyPDFDirectoryLoader("/content/data").load():


    metadata ={
        "source": doc.metadata["source"],
        "page": doc.metadata["page"],

    }
    doc_text = ' '.join(doc.page_content.split())
    # Remove newlines and extra spaces
    docs.append(Document(page_content=doc_text,
                                   metadata=metadata))

# PyPDFDirectoryLoader loads all PDFs in a directory

In [13]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACETOKEN')


In [14]:
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
#from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os
from langchain_chroma import Chroma
from FlagEmbedding import BGEM3FlagModel
# The splitting and chunking strategy
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

# load_dotenv(".env")
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv('HUGGINGFACE_TOKEN')

In [15]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [16]:
# Download the embeddings
open_ai_large = OpenAIEmbeddings(model="text-embedding-3-large")
open_ai_small = OpenAIEmbeddings(model="text-embedding-3-small")


model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
def create_chroma_db(docs, embedding_list, embedding_names, similarity_type="cosine"):
  splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
  chunked_docs = splitter.split_documents(docs)

  for embedding, name in zip(embedding_list, embedding_names):
    for similarity in similarity_type:
      persist_dir = f"./capstone_db_{name}_{similarity}"
      chromba_db = Chroma.from_documents(documents=chunked_docs,
                                        collection_name=f'capstone_db_{name}_{similarity}',
                                        embedding=embedding,
                                        collection_metadata={"hnsw:space": similarity},
                                         persist_directory=persist_dir)

      print(f"Chroma DB created for embedding: {name} with similarity: {similarity} in directory: {persist_dir}")

In [18]:
docs = docs
embedding_list = [open_ai_large, open_ai_small, hf]
embedding_names = ["open_ai_large", "open_ai_small","mp_allnet"]
similarity_type = ["cosine", "l2"]

In [19]:
create_chroma_db(docs=docs, embedding_list=embedding_list, embedding_names=embedding_names, similarity_type=similarity_type)

Chroma DB created for embedding: open_ai_large with similarity: cosine in directory: ./capstone_db_open_ai_large_cosine
Chroma DB created for embedding: open_ai_large with similarity: l2 in directory: ./capstone_db_open_ai_large_l2
Chroma DB created for embedding: open_ai_small with similarity: cosine in directory: ./capstone_db_open_ai_small_cosine
Chroma DB created for embedding: open_ai_small with similarity: l2 in directory: ./capstone_db_open_ai_small_l2
Chroma DB created for embedding: mp_allnet with similarity: cosine in directory: ./capstone_db_mp_allnet_cosine
Chroma DB created for embedding: mp_allnet with similarity: l2 in directory: ./capstone_db_mp_allnet_l2


In [20]:
# embedding_name = []
# similarity_types = []
# retriever_details = []

In [21]:
# for embedding, name in zip(embedding_list, embedding_names):

#   for similarity  in similarity_type:

#     chroma_db = Chroma(persist_directory=f"./capstone_db_{name}_{similarity}",
#                         collection_name=f'capstone_db_{name}_{similarity}',
#                          embedding_function=embedding)

#     similarity_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
#                                                 search_kwargs={"k": 5, "score_threshold": 0.0})
#     embed_search = similarity_retriever.invoke("what is attention layer", k=5)
#     embedding_name.append(name)

#     retriever_details.append(embed_search)
#     similarity_type.append(similarity)


# print((embedding_name, similarity_type))

In [22]:
embedding_list = [open_ai_large, open_ai_small, hf]
embedding_names = ["open_ai_large", "open_ai_small","mp_allnet"]
similarity_type = ["cosine", "l2"]

In [23]:
# embedding_names =[]
# retriever_details = []
# similarity_types = []
combined_list_results = []

def find_best_embedding(embedding_list, embedding_names, similarity_type, question):
  # load from disk
  results = []

  for embedding, name in zip(embedding_list, embedding_names):
    for similarity  in similarity_type:
      print(embedding)
      print(name)
      print(similarity)

      chroma_db = Chroma(persist_directory=f"./capstone_db_{name}_{similarity}",
                        collection_name=f'capstone_db_{name}_{similarity}',
                         embedding_function=embedding)

      similarity_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                                                search_kwargs={"k": 5, "score_threshold": 0.0})
      embed_search = similarity_retriever.invoke(question, k=5)

    combined_list_results.append({
        "embedding_name": name,
        "similarity_type": similarity,
        "retriever_details": embed_search
    })

  return combined_list_results

## Analyze Cosine  Similarity

In [24]:
ques="what is attention layer"
output  = find_best_embedding(embedding_list = embedding_list, embedding_names = embedding_names,
                              similarity_type = similarity_type, question = ques)
print(output)

client=<openai.resources.embeddings.Embeddings object at 0x7de96a036150> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7de967df3750> model='text-embedding-3-large' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True
open_ai_large
cosine
client=<openai.resources.embeddings.Embeddings object at 0x7de96a036150> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7de967df3750> model='text-embedding-3-large' dimensions=

## Analyzing Cosine Similarity results

In [25]:
#Iterate through the list of dictionaries returned by find_best_embedding
for item in output:
    embedding_name = item["embedding_name"]
    similarity_search = "cosine"  #item["similarity_type"] # Renamed to avoid conflict and be more descriptive
    retriever = item["retriever_details"]

    print(f"Embedding: {embedding_name}")
    print(f"Similarity_search: {similarity_search}")
    for doc in retriever:
        print(f"Document: {doc.page_content}")
        #print(f"Score: {doc.metadata['similarity_score']}") # Uncomment if score is available
        print(f"Source: {doc.metadata['source']}")
        print(f"Page: {doc.metadata['page']}")
        print("-" * 80)

Embedding: open_ai_large
Similarity_search: cosine
Document: layer connects all positions with a constant number of sequentially executed operations, whereas a recurrent layer requires O(n) sequential operations. In terms of computational complexity, self-attention layers are faster than recurrent layers when the sequence 6
Source: /content/data/attention_paper.pdf
Page: 5
--------------------------------------------------------------------------------
Document: of Attention in our Model The Transformer uses multi-head attention in three different ways: • In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [38, 2, 9]. • The encoder contains self-attention layers. In a self-attention layer all

In [26]:
#Iterate through the list of dictionaries returned by find_best_embedding
for item in output:
    embedding_name = item["embedding_name"]
    similarity_search = "l2"  #item["similarity_type"] # Renamed to avoid conflict and be more descriptive
    retriever = item["retriever_details"]

    print(f"Embedding: {embedding_name}")
    print(f"Similarity_search: {similarity_search}")
    for doc in retriever:
        print(f"Document: {doc.page_content}")
        #print(f"Score: {doc.metadata['similarity_score']}") # Uncomment if score is available
        print(f"Source: {doc.metadata['source']}")
        print(f"Page: {doc.metadata['page']}")
        print("-" * 80)

Embedding: open_ai_large
Similarity_search: l2
Document: layer connects all positions with a constant number of sequentially executed operations, whereas a recurrent layer requires O(n) sequential operations. In terms of computational complexity, self-attention layers are faster than recurrent layers when the sequence 6
Source: /content/data/attention_paper.pdf
Page: 5
--------------------------------------------------------------------------------
Document: of Attention in our Model The Transformer uses multi-head attention in three different ways: • In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [38, 2, 9]. • The encoder contains self-attention layers. In a self-attention layer all of 

## Cosine similarity is better than Euclidean similarity.

In [27]:
ques="what is gemini"
output  = find_best_embedding(embedding_list = embedding_list, embedding_names = embedding_names,
                              similarity_type = similarity_type, question = ques)
#print(output)

client=<openai.resources.embeddings.Embeddings object at 0x7de96a036150> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7de967df3750> model='text-embedding-3-large' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True
open_ai_large
cosine
client=<openai.resources.embeddings.Embeddings object at 0x7de96a036150> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7de967df3750> model='text-embedding-3-large' dimensions=

In [28]:
#Iterate through the list of dictionaries returned by find_best_embedding
for item in output:
    embedding_name = item["embedding_name"]
    similarity_search = "cosine"  #item["similarity_type"] # Renamed to avoid conflict and be more descriptive
    retriever = item["retriever_details"]

    print(f"Embedding: {embedding_name}")
    print(f"Similarity_search: {similarity_search}")
    for doc in retriever:
        print(f"Document: {doc.page_content}")
        #print(f"Score: {doc.metadata['similarity_score']}") # Uncomment if score is available
        print(f"Source: {doc.metadata['source']}")
        print(f"Page: {doc.metadata['page']}")
        print("-" * 80)

Embedding: open_ai_large
Similarity_search: cosine
Document: layer connects all positions with a constant number of sequentially executed operations, whereas a recurrent layer requires O(n) sequential operations. In terms of computational complexity, self-attention layers are faster than recurrent layers when the sequence 6
Source: /content/data/attention_paper.pdf
Page: 5
--------------------------------------------------------------------------------
Document: of Attention in our Model The Transformer uses multi-head attention in three different ways: • In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [38, 2, 9]. • The encoder contains self-attention layers. In a self-attention layer all

In [29]:
ques="what are the conclusions of llms"
output  = find_best_embedding(embedding_list = embedding_list, embedding_names = embedding_names,
                              similarity_type = similarity_type, question = ques)
#print(output)

client=<openai.resources.embeddings.Embeddings object at 0x7de96a036150> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7de967df3750> model='text-embedding-3-large' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True
open_ai_large
cosine
client=<openai.resources.embeddings.Embeddings object at 0x7de96a036150> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7de967df3750> model='text-embedding-3-large' dimensions=

In [30]:
#Iterate through the list of dictionaries returned by find_best_embedding
for item in output:
    embedding_name = item["embedding_name"]
    similarity_search = "cosine"  #item["similarity_type"] # Renamed to avoid conflict and be more descriptive
    retriever = item["retriever_details"]

    print(f"Embedding: {embedding_name}")
    print(f"Similarity_search: {similarity_search}")
    for doc in retriever:
        print(f"Document: {doc.page_content}")
        #print(f"Score: {doc.metadata['similarity_score']}") # Uncomment if score is available
        print(f"Source: {doc.metadata['source']}")
        print(f"Page: {doc.metadata['page']}")
        print("-" * 80)

Embedding: open_ai_large
Similarity_search: cosine
Document: layer connects all positions with a constant number of sequentially executed operations, whereas a recurrent layer requires O(n) sequential operations. In terms of computational complexity, self-attention layers are faster than recurrent layers when the sequence 6
Source: /content/data/attention_paper.pdf
Page: 5
--------------------------------------------------------------------------------
Document: of Attention in our Model The Transformer uses multi-head attention in three different ways: • In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [38, 2, 9]. • The encoder contains self-attention layers. In a self-attention layer all

## Creating a RAG based model

In [31]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.docstore.document import Document
from langchain import hub
from langchain.chains import create_history_aware_retriever
from langchain_core.messages import HumanMessage, AIMessage

In [32]:
open_ai_large = OpenAIEmbeddings(model="text-embedding-3-large")

In [33]:
docs= []
for doc in PyPDFDirectoryLoader("/content/data").load():


    metadata ={
        "source": doc.metadata["source"],
        "page": doc.metadata["page"],

    }
    doc_text = ' '.join(doc.page_content.split())
    # Remove newlines and extra spaces
    docs.append(Document(page_content=doc_text,
                                   metadata=metadata))

# PyPDFDirectoryLoader loads all PDFs in a directory

In [34]:
# The vectorstore we'll be using
from langchain_chroma import Chroma

# The embedding engine that will convert our text to vectors
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [35]:
splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=300)
chunked_docs = splitter.split_documents(docs)

Create Chroma DB and embedding

In [36]:
chroma_db = Chroma.from_documents(documents=chunked_docs,
                                  collection_name='capstone_db_rag',
                                  collection_metadata={"hnsw:space":"cosine"},
                                  persist_directory="./capstone_db_rag",
                                  embedding=open_ai_large)

In [37]:
similarity_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                                             search_kwargs={"k": 5, "score_threshold": 0.2})

In [38]:
# similarity_retriever.invoke("what is gemini", k=5)

In [39]:
prompt = """You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If the answer is not present in the context, just say that you don't know.
            Keep the answer to the point. also show the top 3 context documents of the answer.

            Question:
            {question}

            Context:
            {context}

            Answer:
         """

prompt_template = ChatPromptTemplate.from_template(prompt)

In [40]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


In [41]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = (
    {
        "context": (similarity_retriever
                      |
                    format_docs),
        "question": RunnablePassthrough()
    }
       |
    prompt_template
      |
    chatgpt
)

In [42]:
from IPython.display import display, Markdown
query = "what is a LLM"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))


A LLM, or Large Language Model, is a type of artificial intelligence model designed to understand and generate human-like text based on the input it receives. These models are characterized by their large number of parameters, which allow them to perform a variety of language tasks effectively.

Top 3 context documents:
1. "Mistral 7B outperforms the best open 13B model (Llama 2) across all evaluated benchmarks..."
2. "In this context, the search for balanced models delivering both high-level performance and efficiency becomes critically essential."
3. "Mistral 7B leverages grouped-query attention (GQA) and sliding window attention (SWA)..."

In [43]:
rephrase_system_prompt = """Given a chat history and the latest user question
which might reference context in the chat history, formulate a standalone question
which can be understood without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is."""

rephrase_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", rephrase_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    chatgpt, similarity_retriever, rephrase_prompt
)


In [44]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks.
                      Use the following pieces of retrieved context to answer the question.
                      If you don't know the answer, just say that you don't know.
                      Keep the answer upto 5 lines unless the user asks for more information

                      Context:
                      {context}
                  """

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(chatgpt, qa_prompt)

qa_rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [45]:
chat_history = []

question = "what is mistral 7b?"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
print(response['answer'])

Mistral 7B is a 7-billion-parameter language model designed for high performance and efficiency in natural language processing tasks. It outperforms other models, including the Llama 2 13B, across various benchmarks. Mistral 7B utilizes advanced attention mechanisms like grouped-query attention and sliding window attention to enhance inference speed and handle longer sequences effectively. It is released under the Apache 2.0 license and is suitable for a wide range of applications.


In [46]:
chat_history.extend([HumanMessage(content=question),
                     AIMessage(content=response['answer'])])

chat_history

[HumanMessage(content='what is mistral 7b?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Mistral 7B is a 7-billion-parameter language model designed for high performance and efficiency in natural language processing tasks. It outperforms other models, including the Llama 2 13B, across various benchmarks. Mistral 7B utilizes advanced attention mechanisms like grouped-query attention and sliding window attention to enhance inference speed and handle longer sequences effectively. It is released under the Apache 2.0 license and is suitable for a wide range of applications.', additional_kwargs={}, response_metadata={})]

In [47]:
question = "Tell me more about Mistral"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
display(Markdown(response['answer']))

Mistral 7B is engineered to balance high performance with efficiency, making it suitable for real-world applications. It surpasses the Llama 2 13B model in reasoning, mathematics, and code generation, while also approaching the coding performance of Code-Llama 7B. The model employs grouped-query attention (GQA) for faster inference and sliding window attention (SWA) to manage longer sequences with reduced computational costs. Additionally, Mistral 7B can be fine-tuned for various tasks and is designed for easy deployment on platforms like AWS, GCP, and Azure.

In [48]:
question = "how is it different from Gemini"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
display(Markdown(response['answer']))

Mistral 7B and Gemini differ primarily in their architecture and capabilities. Mistral 7B focuses on high performance and efficiency in language tasks, utilizing techniques like sliding window attention for improved inference. In contrast, Gemini is a family of multimodal models that advance capabilities across text, code, image, audio, and video, with a broader range of applications. Additionally, Gemini includes models like Gemini Nano, which are optimized for on-device deployments, while Mistral 7B is primarily designed for cloud and local deployment.

In [49]:
question = "how is it different from gpt"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
display(Markdown(response['answer']))

Mistral 7B differs from GPT models primarily in its architecture and design focus. While both are transformer-based language models, Mistral 7B employs unique attention mechanisms like grouped-query attention and sliding window attention, which enhance efficiency and performance, especially for longer sequences. Additionally, Mistral 7B is specifically engineered to balance high performance with lower computational costs, making it more accessible for real-world applications compared to some larger GPT models.

## Multi-user Conversational RAG system with LangChain

In [50]:
# you can run this only when you want to remove all conversation histories
#!rm memory.db

In [51]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI




In [52]:
chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

######### REPHRASER ############
rephrase_system_prompt = """Given a chat history and the latest user question
which might reference context in the chat history, formulate a standalone question
which can be understood without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is."""


In [53]:
rephrase_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", rephrase_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [54]:
history_aware_retriever = create_history_aware_retriever(
    chatgpt, similarity_retriever, rephrase_prompt
)

In [55]:
######### MULTI_USER RAG RESPONSE GENERATOR ############
qa_system_prompt = """You are an assistant for question-answering tasks.
                      Use the following pieces of retrieved context to answer the question.
                      If you don't know the answer, just say that you don't know.
                      Keep the answer upto 5 lines unless the user asks for more information.
                      Also, search for the answer from RAG only


                      Context:
                      {context}
                  """

In [56]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [57]:
# used to retrieve conversation history from database
# based on a specific user or session ID
def get_session_history_db(session_id, topk_conversations=2):
    history = SQLChatMessageHistory(session_id, "sqlite:///memory.db")
    #history.messages = history.messages[-2*topk_conversations:]
    return history

In [58]:
# subset historical conversations
def memory_buffer_window(messages, topk_conversations=2): # each conversation has 2 messages - (human prompt, AI response)
    return messages[-(topk_conversations*2):]

In [59]:
# custom RAG chain which looks at last K conversational messages
question_answer_chain = (
    RunnablePassthrough.assign(chat_history=lambda x: memory_buffer_window(x["chat_history"]),
                                context=lambda x: format_docs(x["context"])
                               )
      |
    qa_prompt
       |
    chatgpt
      |
    StrOutputParser()
)
qa_rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [60]:

############ CONVERSATIONAL RAG CHAIN ####################
conversational_rag_chain = RunnableWithMessageHistory(
    qa_rag_chain,
    get_session_history_db,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [61]:
from IPython.display import display, Markdown

def conv_rag_chatbot(usersession_id, prompt):
    response = conversational_rag_chain.invoke(
                                {"input": prompt},
                                config={
                                    "configurable": {"session_id": usersession_id}
                                }
    )
    print('Answer:')
    display(Markdown(response['answer']))
    print('Sources:')
    for document in response['context']:
        print(document)
        print()

    return response

In [62]:
us_id = 'vij01'
r = conv_rag_chatbot(us_id, 'what is gpt')

/usr/local/lib/python3.11/dist-packages/langchain_core/runnables/history.py:596: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  message_history = self.get_session_history(


Answer:


GPT, or Generative Pre-trained Transformer, is a type of large-scale language model developed by OpenAI. It uses deep learning techniques to understand and generate human-like text based on the input it receives. GPT models are pre-trained on diverse datasets and can perform various tasks, including text generation, summarization, translation, and answering questions. The latest version, GPT-4, is multimodal, meaning it can process both text and images.

Sources:
page_content='Exam GPT-4 GPT-4 (no vision) GPT-3.5 Uniform Bar Exam (MBE+MEE+MPT) 298 / 400 (~90th) 298 / 400 (~90th) 213 / 400 (~10th) LSAT 163 (~88th) 161 (~83rd) 149 (~40th) SAT Evidence-Based Reading & Writing 710 / 800 (~93rd) 710 / 800 (~93rd) 670 / 800 (~87th) SAT Math 700 / 800 (~89th) 690 / 800 (~89th) 590 / 800 (~70th) Graduate Record Examination (GRE) Quantitative 163 / 170 (~80th) 157 / 170 (~62nd) 147 / 170 (~25th) Graduate Record Examination (GRE) Verbal 169 / 170 (~99th) 165 / 170 (~96th) 154 / 170 (~63rd) Graduate Record Examination (GRE) Writing 4 / 6 (~54th) 4 / 6 (~54th) 4 / 6 (~54th) USABO Semifinal Exam 2020 87 / 150 (99th - 100th) 87 / 150 (99th - 100th) 43 / 150 (31st - 33rd) USNCO Local Section Exam 2022 36 / 60 38 / 60 24 / 60 Medical Knowledge Self-Assessment Program 75 % 75 % 53 % Codeforces Rating 392 (below 5th) 392 (below 5th) 260 (below 5th) AP Art History 5 (86th - 100th) 5 (86th - 100th) 5 (86th - 100th) AP Biology 5 (85th - 100th) 5 (85th - 10

In [63]:
us_id = 'vij01'
r = conv_rag_chatbot(us_id, 'what is the difference with mistral')

Answer:


Mistral 7B is a specific language model that is engineered for superior performance and efficiency, outperforming models like Llama 2 13B across various benchmarks. It utilizes unique attention mechanisms such as grouped-query attention (GQA) and sliding window attention (SWA) to enhance inference speed and handle longer sequences effectively. In contrast, GPT models, like those from OpenAI, are designed for general text generation tasks and may not have the same architectural optimizations as Mistral 7B.

Sources:
page_content='Mistral 7B is released under the Apache 2.0 license. This release is accompanied by a reference implementation1 facilitating easy deployment either locally or on cloud platforms such as AWS, GCP, or Azure using the vLLM [17] inference server and SkyPilot 2. Integration with Hugging Face 3 is also streamlined for easier integration. Moreover, Mistral 7B is crafted for ease of fine-tuning across a myriad of tasks. As a demonstration of its adaptability and superior performance, we present a chat model fine-tuned from Mistral 7B that significantly outperforms the Llama 2 13B – Chat model. Mistral 7B takes a significant step in balancing the goals of getting high performance while keeping large language models efficient. Through our work, our aim is to help the community create more affordable, efficient, and high-performing language models that can be used in a wide range of real-world applications. 2 Architectural details Figure 1: Sliding Window Attention. The num

In [64]:
us_id = 'vij01'
r = conv_rag_chatbot(us_id, 'what is mistral')

Answer:


Mistral 7B is a 7-billion-parameter language model designed for high performance and efficiency in natural language processing tasks. It outperforms other models, such as Llama 2 13B, in various benchmarks, particularly in reasoning, mathematics, and code generation. Mistral 7B employs advanced attention mechanisms like grouped-query attention (GQA) and sliding window attention (SWA) to improve inference speed and manage longer sequences effectively. It is released under the Apache 2.0 license and is suitable for fine-tuning across multiple tasks.

Sources:
page_content='Mistral 7B Albert Q. Jiang, Alexandre Sablayrolles, Arthur Mensch, Chris Bamford, Devendra Singh Chaplot, Diego de las Casas, Florian Bressand, Gianna Lengyel, Guillaume Lample, Lucile Saulnier, Lélio Renard Lavaud, Marie-Anne Lachaux, Pierre Stock, Teven Le Scao, Thibaut Lavril, Thomas Wang, Timothée Lacroix, William El Sayed Abstract We introduce Mistral 7B, a 7–billion-parameter language model engineered for superior performance and efficiency. Mistral 7B outperforms the best open 13B model (Llama 2) across all evaluated benchmarks, and the best released 34B model (Llama 1) in reasoning, mathematics, and code generation. Our model leverages grouped-query attention (GQA) for faster inference, coupled with sliding window attention (SW A) to effectively handle sequences of arbitrary length with a reduced inference cost. We also provide a model fine-tuned to follow instructions, Mistral 7B – Instruct, that surpasses Llama 2 13B – chat model both on human and autom

In [72]:
us_id = 'vij02'
r = conv_rag_chatbot(us_id, 'what is instructgpt')

Answer:


InstructGPT is a model developed by OpenAI that is specifically fine-tuned to follow user instructions more effectively than previous models like GPT-3. It utilizes a combination of supervised learning and reinforcement learning from human feedback (RLHF) to align its outputs with user preferences. InstructGPT demonstrates improvements in truthfulness, reliability, and the ability to handle a variety of tasks, such as question answering and summarization.

Sources:
page_content='To train the very ﬁrst InstructGPT models, we asked labelers to write prompts themselves. This is because we needed an initial source of instruction-like prompts to bootstrap the process, and these kinds of prompts weren’t often submitted to the regular GPT-3 models on the API. We asked labelers to write three kinds of prompts: • Plain: We simply ask the labelers to come up with an arbitrary task, while ensuring the tasks had sufﬁcient diversity. • Few-shot: We ask the labelers to come up with an instruction, and multiple query/response pairs for that instruction. • User-based: We had a number of use-cases stated in waitlist applications to the OpenAI API. We asked labelers to come up with prompts corresponding to these use cases. From these prompts, we produce three different datasets used in our ﬁne-tuning procedure: (1) our SFT dataset, with labeler demonstrations used to train our SFT models, (2) our RM dataset, with labeler rankings of model outputs used to 

In [71]:
us_id = 'vij02'
r = conv_rag_chatbot(us_id, 'how is instructgpt different from other large language models')

Answer:


InstructGPT differs from other large language models like GPT-3 primarily in its training approach. While GPT-3 is trained on a general language modeling objective, InstructGPT is fine-tuned using human feedback to better follow user instructions. This results in InstructGPT generating outputs that are more aligned with user intent, showing improvements in truthfulness, reduced toxicity, and better adherence to explicit constraints. Additionally, InstructGPT has been shown to outperform GPT-3 in user preference evaluations despite having fewer parameters.

Sources:
page_content='Figure 2: A diagram illustrating the three steps of our method: (1) supervised ﬁne-tuning (SFT), (2) reward model (RM) training, and (3) reinforcement learning via proximal policy optimization (PPO) on this reward model. Blue arrows indicate that this data is used to train one of our models. In Step 2, boxes A-D are samples from our models that get ranked by labelers. See Section 3 for more details on our method. sizes (1.3B, 6B, and 175B parameters), and all of our models use the GPT-3 architecture. Our main ﬁndings are as follows: Labelers signiﬁcantly prefer InstructGPT outputs over outputs from GPT-3. On our test set, outputs from the 1.3B parameter InstructGPT model are preferred to outputs from the 175B GPT-3, despite having over 100x fewer parameters. These models have the same architecture, and differ only by the fact that InstructGPT is ﬁne-tuned on our human data. This result holds true even when we add a few-shot prompt to GPT-3 to make it better at fo

In [75]:
us_id = 'vij02'
r = conv_rag_chatbot(us_id, 'how does instructgpt different from other LLMs')

Answer:


InstructGPT differs from other large language models (LLMs) primarily in its training methodology. It is fine-tuned using human feedback to better align with user instructions, whereas many LLMs, like GPT-3, are trained on a general language modeling objective. This fine-tuning allows InstructGPT to generate more helpful, truthful, and less toxic outputs. Additionally, InstructGPT has been shown to outperform other models in user preference evaluations, even with fewer parameters.

Sources:
page_content='Figure 2: A diagram illustrating the three steps of our method: (1) supervised ﬁne-tuning (SFT), (2) reward model (RM) training, and (3) reinforcement learning via proximal policy optimization (PPO) on this reward model. Blue arrows indicate that this data is used to train one of our models. In Step 2, boxes A-D are samples from our models that get ranked by labelers. See Section 3 for more details on our method. sizes (1.3B, 6B, and 175B parameters), and all of our models use the GPT-3 architecture. Our main ﬁndings are as follows: Labelers signiﬁcantly prefer InstructGPT outputs over outputs from GPT-3. On our test set, outputs from the 1.3B parameter InstructGPT model are preferred to outputs from the 175B GPT-3, despite having over 100x fewer parameters. These models have the same architecture, and differ only by the fact that InstructGPT is ﬁne-tuned on our human data. This result holds true even when we add a few-shot prompt to GPT-3 to make it better at fo

In [76]:
us_id = 'vij02'
r = conv_rag_chatbot(us_id, 'is the above model same as gpt4')

Answer:


InstructGPT is not the same as GPT-4, but it shares some foundational technology. InstructGPT is a specific fine-tuned version of earlier models like GPT-3, designed to follow instructions better. GPT-4, on the other hand, is a more advanced, multimodal model that can process both text and image inputs, exhibiting improved performance across various benchmarks and tasks compared to its predecessors, including InstructGPT.

Sources:
page_content='GPT-4 Technical Report OpenAI∗ Abstract We report the development of GPT-4, a large-scale, multimodal model which can accept image and text inputs and produce text outputs. While less capable than humans in many real-world scenarios, GPT-4 exhibits human-level performance on various professional and academic benchmarks, including passing a simulated bar exam with a score around the top 10% of test takers. GPT-4 is a Transformer- based model pre-trained to predict the next token in a document. The post-training alignment process results in improved performance on measures of factuality and adherence to desired behavior. A core component of this project was developing infrastructure and optimization methods that behave predictably across a wide range of scales. This allowed us to accurately predict some aspects of GPT-4’s performance based on models trained with no more than 1/1,000th the compute of GPT-4. 1 Introduction This technical report presents GPT-4, a larg

In [65]:
from selfcheckgpt.modeling_selfcheck_apiprompt import SelfCheckAPIPrompt

In [66]:
sfca = SelfCheckAPIPrompt(client_type="openai", model="gpt-4o")

Initiate OpenAI client... model = gpt-4o


## Identify whether the model hallucinates


1.   First extract the relevant sentences from the paper manually and save it as a list called "sentences"
2.   Extract the relevant output from the model and save it as "samples"
3.   Run it through self check gpt.  Lower the score, better it is





In [73]:
sentences = ["We focus on fine-tuning approaches to aligning language models. Specifically, we use reinforcement\
learning from human feedback (RLHF; Christiano et al., 2017; Stiennon et al., 2020) to fine-tune\
GPT-3 to follow a broad class of written instructions (see Figure 2). This technique uses human\
preferences as a reward signal to fine-tune our models. We first hire a team of 40 contractors to label\
our data, based on their performance on a screening test (see Section 3.4 and Appendix B.1 for more\
details). We then collect a dataset of human-written demonstrations of the desired output behavior\
on (mostly English) prompts submitted to the OpenAI API3 and some labeler-written prompts, and\
use this to train our supervised learning baselines. Next, we collect a dataset of human-labeled\
comparisons between outputs from our models on a larger set of API prompts. We then train a reward\
model (RM) on this dataset to predict which model output our labelers would prefer. Finally, we\
use this RM as a reward function and fine-tune our supervised learning baseline to maximize this\
reward using the PPO algorithm (Schulman et al., 2017). We illustrate this process in Figure 2. This\
procedure aligns the behavior of GPT-3 to the stated preferences of a specific group of people (mostly\
our labelers and researchers), rather than any broader notion of “human values”; we discuss this\
further in Section 5.2. We call the resulting models InstructGPT.\
"]

samples = [
        "InstructGPT is a model developed by OpenAI that is fine-tuned to follow instructions \
        more effectively than previous models like GPT-3. It is trained using a combination of \
        supervised learning and reinforcement learning from human feedback (RLHF) to align its \
        outputs with user preferences. InstructGPT shows improvements in truthfulness, reliability,\
         and the ability to handle diverse tasks, including question answering and summarization.",

        "InstructGPT is a model developed by OpenAI that is specifically fine-tuned to follow user \
        instructions more effectively than previous models like GPT-3. It utilizes a combination of\
         supervised learning and reinforcement learning from human feedback (RLHF) to align its outputs\
          with user preferences. InstructGPT demonstrates improvements in truthfulness, reliability,\
           and the ability to handle a variety of tasks, such as question answering and summarization."
]

In [74]:
sfca.predict(sentences=sentences, sampled_passages=samples)

array([0.])

No hallucination found.  Try it with second set of data

In [83]:
sentences = ["InstructGPT shows small improvements in toxicity over GPT-3, but not bias. To measure\
toxicity, we use the RealToxicityPrompts dataset (Gehman et al., 2020) and conduct both automatic\
and human evaluations. InstructGPT models generate about 25% fewer toxic outputs than GPT-3\
when prompted to be respectful. InstructGPT does not significantly improve over GPT-3 on the\
Winogender (Rudinger et al., 2018) and CrowSPairs (Nangia et al., 2020) datasets.\
Our results are in Figure 7. We find that, when instructed to produce a safe and respectful output\
(“respectful prompt”), InstructGPT models generate less toxic outputs than those from GPT-3\
according to the Perspective API. This advantage disappears when the respectful prompt is removed\
(“no prompt”). Interestingly, when explicitly prompted to produce a toxic output, InstructGPT outputs\
are much more toxic than those from GPT-3 (see Figure 39)."
]

In [79]:
us_id = 'vij02'
r = conv_rag_chatbot(us_id, 'what is the improvement shown by instructggpt compared to gpt3 in toxicity')

Answer:


InstructGPT generates about 25% fewer toxic outputs than GPT-3 when prompted to be respectful. However, this advantage diminishes when the respectful prompt is not used, indicating that the context of the prompt plays a crucial role in the model's toxicity levels.

Sources:
page_content='distribution. In a head to head comparison, our 175B InstructGPT model outputs were preferred over our FLAN model 78 ±4% of the time and over our T0 model 79 ±4% of the time. Likert scores for these models are shown in Figure 5. We believe our InstructGPT model outperforms FLAN and T0 for two reasons. First, public NLP datasets are designed to capture tasks that are easy to evaluate with automatic metrics, such as classiﬁcation, question answering, and to a certain extent summarization and translation. However, classiﬁcation and QA are only a small part (about 18%) of what API customers use our language models for, whereas open-ended generation and brainstorming consist of about 57% of our prompt dataset according to labelers (see Table 1). Second, it can be difﬁcult for public NLP datasets to obtain a very high diversity of inputs (at least, on the kinds of inputs that real-world users would be interested in using). Of course, tasks found in NLP datasets do repr

In [80]:
samples = ["InstructGPT shows a reduction in toxicity compared to GPT-3, generating about 25% \
fewer toxic outputs when prompted to be respectful. However, when the respectful prompt is removed, \
the advantage in toxicity disappears, indicating that the context of the prompt significantly influences the output.",
"InstructGPT generates about 25% fewer toxic outputs than GPT-3 when\
prompted to be respectful. However, this advantage diminishes when the respectful prompt is not used,\
indicating that the context of the prompt plays a crucial role in the model's toxicity levels."]

No hallucination of data